In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your dataset
df = pd.read_csv('/content/Financial-QA-10k.csv')

# Verify structure
print(df.columns)  # Should have: question, answer, context, ticker, filing
print(df.head())

# Convert to Hugging Face Dataset format
from datasets import Dataset
dataset = Dataset.from_pandas(df)

# Split into train/test
train_test = dataset.train_test_split(test_size=0.2)
train_dataset = train_test["train"]
eval_dataset = train_test["test"]

Index(['question', 'answer', 'context', 'ticker', 'filing'], dtype='object')
                                            question  \
0  What area did NVIDIA initially focus on before...   
1  What are some of the recent applications of GP...   
2  What significant invention did NVIDIA create i...   
3  How does NVIDIA's platform strategy contribute...   
4  What does NVIDIA's CUDA programming model enable?   

                                              answer  \
0           NVIDIA initially focused on PC graphics.   
1  Recent applications of GPU-powered deep learni...   
2                   NVIDIA invented the GPU in 1999.   
3  NVIDIA's platform strategy brings together har...   
4  NVIDIA's CUDA programming model opened the par...   

                                             context ticker    filing  
0  Since our original focus on PC graphics, we ha...   NVDA  2023_10K  
1  Some of the most recent applications of GPU-po...   NVDA  2023_10K  
2  Our invention of the GPU in 19

In [3]:
pip install transformers[torch] datasets accelerate sentencepiece peft bitsandbytes gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 838.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_checkpoint = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:
def preprocess_function(examples):
    # Ensure all elements are strings and handle potential missing values
    questions = [str(q) for q in examples["question"]]
    contexts = [str(c) for c in examples["context"]]
    answers = [str(a) for a in examples["answer"]]

    # Create properly formatted inputs
    inputs = [f"question: {q} context: {c}" for q, c in zip(questions, contexts)]

    # Tokenize inputs
    model_inputs = tokenizer(
        inputs,
        max_length=256,
        truncation=True,
        padding="max_length",
        return_tensors="pt"  # Add this to ensure proper tensor format
    )

    # Tokenize targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            answers,
            max_length=64,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing with batched processing
tokenized_train = train_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=32,  # Reduce if you get memory errors
    remove_columns=train_dataset.column_names  # Remove original columns
)

tokenized_eval = eval_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=32,
    remove_columns=eval_dataset.column_names
)

Map:   0%|          | 0/5600 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

In [6]:
# Set training parameters
training_args = Seq2SeqTrainingArguments(
    output_dir="./finetuned-chatbot",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,  # Enable if using GPU
    push_to_hub=False  # Change to True when ready
)

# Create data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Initialize Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-6-2bf355b9d3d5>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [7]:
trainer.train()

# Save model
model.save_pretrained("./finetuned-chatbot")
tokenizer.save_pretrained("./finetuned-chatbot")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yadav-anuj051120 (yadav-anuj051120-iit-bomba) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.000000,nan
2,0.000000,nan
3,0.000000,nan
4,0.000000,nan
5,0.000000,nan


('./finetuned-chatbot/tokenizer_config.json',
 './finetuned-chatbot/special_tokens_map.json',
 './finetuned-chatbot/spiece.model',
 './finetuned-chatbot/added_tokens.json',
 './finetuned-chatbot/tokenizer.json')

In [8]:
from huggingface_hub import notebook_login
notebook_login()  # Run this before training if pushing to Hub

# Then set push_to_hub=True in training_args

In [10]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00


In [12]:
!pip install rouge_score # Install the missing rouge_score package

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=20a9c4505f202ab8327ad3e5b0c0d1521a64d64741b03fa7a4c9983ac901f272
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [21]:
import evaluate
import numpy as np

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Remove predictions outside the valid vocabulary range
    predictions = np.clip(predictions, 0, tokenizer.vocab_size - 1)

    decoded_preds = tokenizer.batch_decode(predictions.tolist(), skip_special_tokens=True)

    # Replace -100 in labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )

    return {k: round(v, 4) for k, v in result.items()}

# Add this to trainer before evaluation
trainer.compute_metrics = compute_metrics

# Evaluate
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': nan, 'eval_rouge1': 0.449, 'eval_rouge2': 0.3458, 'eval_rougeL': 0.4424, 'eval_rougeLsum': 0.4428, 'eval_runtime': 101.1959, 'eval_samples_per_second': 13.835, 'eval_steps_per_second': 3.459, 'epoch': 5.0}


In [16]:
import gradio as gr

def generate_answer(question, context):
    input_text = f"question: {question} context: {context}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=200)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

demo = gr.Interface(
    fn=generate_answer,
    inputs=[
        gr.Textbox(label="Question"),
        gr.Textbox(label="Context (optional)", lines=3)
    ],
    outputs=gr.Textbox(label="Answer"),
    title="Financial Chatbot",
    description="Ask questions about financial data and company updates"
)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7f23945d3dd1cba1ee.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [23]:
from transformers import pipeline

# Load quantized model (4-bit)
quantized_model = AutoModelForSeq2SeqLM.from_pretrained(
    "./finetuned-chatbot",
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.float16
)

# Create optimized pipeline - Remove the device argument
chatbot = pipeline(
    "text2text-generation",
    model=quantized_model,
    tokenizer=tokenizer,
    # device=0 if torch.cuda.is_available() else -1,  # Remove this line
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Device set to use cuda:0


In [24]:
def generate_answer(question, context):
    input_text = f"question: {question} context: {context}"

    return chatbot(
        input_text,
        max_new_tokens=100,          # Reduced from 200
        num_beams=2,                 # Faster than default 4
        early_stopping=True,
        do_sample=False,             # Disable sampling for speed
        temperature=0.7,
        repetition_penalty=1.2
    )[0]['generated_text']

In [25]:
from functools import lru_cache

@lru_cache(maxsize=2)
def load_model():
    return pipeline(...)  # Use same config as above

In [29]:
import gradio as gr
import time  # Import time for delay

demo = gr.Blocks()

with demo:
    gr.Markdown("# Financial Chatbot")
    with gr.Row():
        with gr.Column():
            question = gr.Textbox(label="Question")
            context = gr.Textbox(label="Context (optional)", lines=3)
            btn = gr.Button("Submit")
        with gr.Column():
            answer = gr.Textbox(label="Answer", interactive=False) # Default loading is already enabled when interactive=False
            # The gr.load() call is removed because you want the default loading behavior of Textbox

    btn.click(
        fn=generate_answer,
        inputs=[question, context],
        outputs=answer,
        api_name="predict",
        preprocess=False  # Disable automatic input processing
    ).then(
        lambda: time.sleep(0.1),  # Add small delay for UI update
        None,
        None,
        queue=False
    )

In [30]:
# Add these before inference:
torch.backends.cudnn.benchmark = True
torch.set_float32_matmul_precision('medium')

In [33]:
!pip install diskcache

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00


In [34]:
from diskcache import Cache

cache = Cache("./response_cache")

@cache.memoize(expire=3600)  # Cache responses for 1 hour
def cached_generation(input_text):
    return chatbot(input_text, **generation_params)

In [37]:
from tqdm import tqdm

def generate_with_progress(input_text):
    with tqdm(total=100, desc="Generating") as pbar:
        result = chatbot(input_text)
        pbar.update(100)
    return result

In [43]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from functools import lru_cache
import gradio as gr
import time

# Hardware optimizations
torch.backends.cudnn.benchmark = True
torch.set_float32_matmul_precision('medium')

MODEL_PATH = "./finetuned-chatbot"  # Path to your saved model

@lru_cache(maxsize=1)
def load_components():
    """Load model and tokenizer with caching"""
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

    model = AutoModelForSeq2SeqLM.from_pretrained(
        MODEL_PATH,
        device_map="auto",
        load_in_4bit=True,
        torch_dtype=torch.float16,
        # use_flash_attention_2=True  # Uncomment if supported
    )

    return tokenizer, model

tokenizer, model = load_components()

# Create optimized pipeline - Remove the device argument
chatbot_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    # device=0 if torch.cuda.is_available() else -1,  # Remove or comment out this line
)

def generate_answer(question, context=None):
    """Optimized generation function with error handling"""
    try:
        input_text = f"question: {question}"
        if context:
            input_text += f" context: {context}"

        start_time = time.time()

        result = chatbot_pipeline(
            input_text,
            max_new_tokens=100,
            num_beams=2,
            early_stopping=True,
            do_sample=False,
            temperature=0.7,
            repetition_penalty=1.2
        )[0]['generated_text']

        print(f"Generated in {time.time() - start_time:.2f}s")
        return result

    except Exception as e:
        return f"Error generating answer: {str(e)}"

with gr.Blocks(title="Financial Chatbot") as demo:
    gr.Markdown("# 📈 Financial Chatbot")
    gr.Markdown("Ask questions about company financials and market updates")

    with gr.Row():
        with gr.Column():
            question_input = gr.Textbox(
                label="Your Question",
                placeholder="e.g., Should I invest in NVIDIA?",
                max_lines=3
            )
            context_input = gr.Textbox(
                label="Additional Context (optional)",
                placeholder="Paste relevant news/article text here",
                lines=5
            )
            submit_btn = gr.Button("Get Analysis", variant="primary")

        with gr.Column():
            answer_output = gr.Textbox(
                label="Expert Analysis",
                interactive=False,
                show_copy_button=True,
                visible=True  # Start as visible
            )
            # The gr.Loading() component is removed since it's not needed here
            # Loading behavior is handled automatically by Gradio when the output
            # component is set to interactive=False

    # Setup interactions
    submit_event = submit_btn.click(
        fn=lambda: [
            gr.update(visible=False),  # Hide answer box
            gr.update(value="Analyzing...", visible=True)  # Show loading message
        ],
        outputs=[answer_output, answer_output],
        queue=True
    ).then(
        fn=generate_answer,
        inputs=[question_input, context_input],
        outputs=answer_output
    ).then(
        fn=lambda result: gr.update(value=result, visible=True),
        inputs=[answer_output],
        outputs=[answer_output]
    )

    # Example questions
    gr.Examples(
        examples=[
            ["What is NVIDIA's current market strategy?"],
            ["Should I buy Apple stocks given recent earnings?"],
            ["How does Tesla's debt ratio compare to industry average?"]
        ],
        inputs=[question_input]
    )

if __name__ == "__main__":
    demo.launch(
        server_name="0.0.0.0",
        share=True,
        # enable_queue=True,  # Remove this line - queuing is handled automatically
        max_threads=4,
        show_error=True
    )

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Device set to use cuda:0


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7c6b2ae5ff6df303d7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
